In [28]:
def randInstanceGen(n,m,J,U):
    import numpy as np
    import math
    # Reset seed for safety
    np.random.seed(0)
    # Set seed
    np.random.seed(61836)
    
    c = np.random.randint(1,U+1,(J,n))
    a = np.random.randint(1,U+1,(n,m))
    
    b = np.max(a, axis=0)
    
    for i in range(0, m):
        if math.ceil(np.sum(a[:,i])/2) > b[i]:
            b[i] = math.ceil(np.sum(a[:,i])/2)
    
    return c*-1, a, b
def rectangular(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    if m.status != 2:
        return [],1
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    if NW == SE:
        return found, numRect
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
     
        #8
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13  
        R3 = [R[0], (tempNW[0] - 1, ((R[0][1] + R[1][1])/2.0) + .001)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1] , name = 'bottom bound')
        
        m.optimize()
        
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
    return found, numRect

def rectangularMarg(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    if m.status != 2:
        return [],1
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    if NW == SE:
        return found, numRect
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
     
        #8
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1]]
        slope = (R[0][1]-R[1][1])/((R[1][0]-R[0][0])*1.0)
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13  
        R3 = [R[0], (tempNW[0] - 1, ((R[0][1] + R[1][1])/2.0) + .001)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1] , name = 'bottom bound')
        
        m.optimize()
        
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
    return found, numRect

In [31]:
c,a,b =randInstanceGen(20,1,2,40)

In [33]:
a,b = rectangular(c,a,b)
len(a)

13